In [1]:
import geopandas as gpd
import os
import pandas as pd

In [2]:
cwd = os.getcwd()
rchpath = "C:/Users/user/OneDrive - Politecnico di Milano/hydrogeo-modelling/Progetti/MAURICE/MAURICE_progetto/Elaborazioni/IdrAgra/modalita_fabbisogni"

In [18]:
nomefile = "field_percolation_noGW_weekly_012023-092024.csv"

In [17]:
centroidi = gpd.read_file(os.path.join(cwd, 'data', 'shp', 'modello_MAURICE_centroidi.shp'))
rch = pd.read_csv(os.path.join(rchpath, nomefile), sep = ',', index_col=None)

# add 'row' and 'column' columns
rch['row'] = [code.split('x')[0] for code in rch.rXc]
rch['column'] = [code.split('x')[1] for code in rch.rXc]
# keep only points present in the MODFLOW model boundaries
rch = rch.loc[~rch.column.isin(['124','125']), :]

# keep sp23 to sp91
rch.drop(columns = [f'SP{i}' for i in range(1,23)], inplace=True)
rch = rch.iloc[:,1:]
rch.columns = rch.columns[0:3].tolist() + [f'SP{i}' for i in range(1,70)]

centroidi['rXc'] = [f'{r}x{c}' for r,c in zip(centroidi.row, centroidi.column)]
centroidi = centroidi.loc[:,['row','column', 'rXc', 'geometry']]

rchshp = centroidi.merge(rch.iloc[:, 2:], on = 'rXc')

rchshp.to_file(os.path.join(rchpath, "shapefile", f"{nomefile.replace('012023', '062023').split('.')[0]}.shp"))